In [1]:
% matplotlib inline
import pandas as pd
import pickle as pkl
import string
import numpy as np; np.random.seed(7)
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
import re
import time

## Data Loadings

### word 2 vec repre

In [2]:
w2v_map = {}
with open('data/vectors_pruned.200.txt', 'r') as src:
    src = src.read().strip().split('\n')
    for line in src:
        wv = line.strip().split(' ')
        word = wv.pop(0)
        w2v_map[word] = np.array(list(map(float, wv)))

In [3]:
w2i_map = {}
for i, key in enumerate(w2v_map.keys()):
    w2i_map[key] = i

In [4]:
pkl.dump(w2i_map, open('data/word_idx_map.pkl','wb'))

### map Q idx to context

In [5]:
w2v_matrix = np.zeros(( len((w2v_map.keys())), 200 ))
counter = 0
for _, val in w2v_map.items():
    w2v_matrix[counter] = val
    counter += 1

In [6]:
pkl.dump(w2i_map, open('data/w2v_matrix.pkl','wb'))

In [7]:
def w2v(w):
    return w2v_matrix[w2i_map[w]]

In [8]:
def sen2w(sen):
    processed = []
    sen = re.sub(r'[!#\'(),/:?\{}]', ' ', sen).strip().split()
    if len(sen) > 100:
        sen = sen[:100]
    for w in sen:
        # ignore date
        if re.match(r'\d{1,}-\d{1,}-\d{1,}', w):
            continue
        if re.match(r'\d{1,}:\d{1,}', w):
            continue
        if w in w2i_map:
            processed += [w]
        else:
            separated = re.findall(r"[^\W\d_]+|\d+|[=`%$\^\-@;\[&_*>\].<~|+\d+]", w)
            if len(set(separated)) == 1:
                continue
            if separated.count('*') > 3 or separated.count('=') > 3:
                continue
            for separate_w in separated:
                if separate_w in w2i_map:
                    processed += [separate_w]
    return processed

In [9]:
# fixed context len = 125
context_repre = {}
with open('data/text_tokenized.txt', 'r') as src:
    src = src.read().strip().split('\n')
    for line in src:
        context = line.strip().split('\t')
        qid = context.pop(0)
        if len(context) == 1:
            context_repre[int(qid)] = {'t': sen2w(context[0]), 'b': None}
        else:
            len_title = len(context[0])
            if len_title >= 125:
                context_repre[int(qid)] = {'t':sen2w(context[0])[:125], 'b': None}
            else:
                context_repre[int(qid)] = {'t':sen2w(context[0]), 'b': sen2w(context[1])[:125 - len_title]}

### len of context ??

In [631]:
all_lens = []
for k, v in context_repre.items():
    t, b = v['t'], v['b']
    if not v['b']:
        b = []
    all_lens += [len(t)+len(b)]

In [633]:
all(np.array(all_lens) < 125)

True

In [10]:
train_df = pd.read_csv('data/train_random.txt', header=None, delimiter='\t', names=['Q','Q+','Q-'])

In [11]:
train_df.head()

,Q,Q+,Q-
0,262144,211039,227387 413633 113297 356390 256881 145638 2962...
1,491522,65911,155119 402211 310669 383107 131731 299465 1633...
2,240299,168608 390642,368007 70009 48077 376760 438005 228888 142340...
3,196614,205184,334471 163710 376791 441664 159963 406360 4300...
4,360457,321532,151863 501857 217578 470017 125838 31836 42066...


In [12]:
def build_set_pair_with_idx(df):
    idx_set = {}
    for idx, row in df.iterrows():
        idx_set[row['Q']] = {'pos': np.array(list(map(int, row['Q+'].split(' ')))), \
                             'neg': np.array(list(map(int, row['Q-'].split(' '))))}
    return idx_set

In [13]:
train_idx_set = build_set_pair_with_idx(train_df)

## Process Batch

In [14]:
def contxt2vec(title, body=None):
    
    if body == None:
        body = []
    
    v = np.zeros( (len(title) + len(body), 200) )
    counter = 0
    
    for t in title:
        v[counter] = w2v(t)
        counter += 1

    for b in body:
        v[counter] = w2v(b)
        counter += 1
    
    return v

In [15]:
def process_batch(qids, idx_set):
    
    total_pos_len = 0
    for qid in qids:
        total_pos_len += len(idx_set[qid]['pos'])
    
    # per batch element x: vstack, [query_Q x 1; pos_Q x 1; neg_Q x 20]
    # per batch element y: vstack, [query_Q=-1; pos_Q=1; neg_Q=0]
    batch_x = np.zeros(( total_pos_len * 22, 124, 200 ))
    seq_len = np.zeros(total_pos_len * 22)
    
    counter = 0
    for qid in qids:
        
        q_title, q_body = context_repre[qid]['t'], context_repre[qid]['b']
        q_pos = idx_set[qid]['pos']

        # usually one sample
        for qid_pos in q_pos:
            title, body = context_repre[qid_pos]['t'], context_repre[qid_pos]['b']
            # query Q
            if not q_body:
                q_seq_len = len(q_title)
            else:
                q_seq_len = len(q_title) + len(q_body)
            seq_len[counter] = q_seq_len
            batch_x[counter, :q_seq_len] = contxt2vec(q_title, q_body)
            counter += 1
            # pos Q
            if not body:
                pos_q_seq_len = len(title)
            else:
                pos_q_seq_len = len(title) + len(body)
            seq_len[counter] = pos_q_seq_len
            batch_x[counter, :pos_q_seq_len] = contxt2vec(title, body)
            counter += 1
        
            q_neg = idx_set[qid]['neg']
            q_neg_sample_indices = np.random.choice(range(100), size=20)
            q_random_neg = q_neg[q_neg_sample_indices]
            # neg Q
            for qid_neg in q_random_neg:
                title, body = context_repre[qid_neg]['t'], context_repre[qid_neg]['b']
                if not body:
                    neg_q_seq_len = len(title)
                else:
                    neg_q_seq_len = len(title) + len(body)
                seq_len[counter] = neg_q_seq_len
                batch_x[counter, : neg_q_seq_len] = contxt2vec(title, body)
                counter += 1
    
    return batch_x, seq_len

## Evaluation

In [40]:
def read_annotations(path, K_neg=20, prune_pos_cnt=10):
    lst = [ ]
    with open(path) as fin:
        for line in fin:
            parts = line.split("\t")
            pid, pos, neg = parts[:3]
            pos = pos.split()
            neg = neg.split()
            if len(pos) == 0 or (len(pos) > prune_pos_cnt and prune_pos_cnt != -1): continue
            if K_neg != -1:
                np.random.shuffle(neg)
                neg = neg[:K_neg]
            s = set()
            qids = [ ]
            qlabels = [ ]
            for q in neg:
                if q not in s:
                    qids.append(q)
                    qlabels.append(0 if q not in pos else 1)
                    s.add(q)
            for q in pos:
                if q not in s:
                    qids.append(q)
                    qlabels.append(1)
                    s.add(q)
            lst.append((pid, qids, qlabels))

    return lst

In [34]:
class Evaluation():
    
    def __init__(self, data):
        self.data = data

    def Precision(self, precision_at):
        scores = []
        for item in self.data:
            temp = item[:precision_at]
            if any(val==1 for val in item):
                scores.append(sum([1 if val==1 else 0 for val in temp])*1.0 / len(temp) if len(temp) > 0 else 0.0)
        return sum(scores)/len(scores) if len(scores) > 0 else 0.0

    def MAP(self):
        scores = []
        missing_MAP = 0
        for item in self.data:
            temp = []
            count = 0.0
            for i,val in enumerate(item):
                if val == 1:
                    count += 1.0
                    temp.append(count/(i+1))
                if len(temp) > 0:
                    scores.append(sum(temp) / len(temp))
                else:
                    missing_MAP += 1
        return sum(scores)/len(scores) if len(scores) > 0 else 0.0

    def MRR(self):
        scores = []
        for item in self.data:
            for i,val in enumerate(item):
                if val == 1:
                    scores.append(1.0/(i+1))
                    break

        return sum(scores)/len(scores) if len(scores) > 0 else 0.0

## Models

In [65]:
class EmbeddingLayer(nn.Module):
    
    def __init__(self, input_size, hidden_size, layer_type):
        
        super(EmbeddingLayer, self).__init__()
        
        self.hidden_size = hidden_size
        
        if layer_type == 'lstm':
            self.embedding_layer = nn.LSTM(input_size, hidden_size)
        elif layer_type == 'cnn':
            pass
        
        self.tanh = nn.Tanh()

    def init_hidden(self, batch_size):
        return (Variable(torch.zeros(batch_size, 1, self.hidden_size)), Variable(torch.zeros(batch_size, 1, self.hidden_size)))

    def forward(self, context, seq_len):
        lstm_out, self.hidden = self.embedding_layer(context, (self.tanh(self.hidden[0]), self.tanh(self.hidden[1])))
        mask = build_mask(seq_len)
        embeddings = torch.sum(lstm_out * Variable(torch.FloatTensor(mask)), dim=1) \
            / Variable(torch.FloatTensor(np.sum(mask, axis=1)))
        return embeddings

In [17]:
def cos_sim(qv, qv_):
    return torch.sum(qv * qv_, dim=1) / (torch.sqrt(torch.sum(qv ** 2, dim=1)) * torch.sqrt(torch.sum(qv_ ** 2, dim=1)))

In [25]:
def criterion(embeddings):
    
    # a batch of embeddings
    num_block = embeddings.size()[0] // 22
    loss = 0
    for i in range(num_block):
        block_embeddings = embeddings[ i * 22: (i + 1) * 22 ]
        qs = block_embeddings[0]
        qs_ = block_embeddings[1:22]
        pos_score = cos_sim(qs.expand(21, 240), qs_)[0]
        neg_score = torch.max(cos_sim(qs.expand(21, 240), qs_)[1:])
        diff = neg_score - pos_score + 1 # margin=1
        if diff.data[0] > 0:
            loss += diff
            
    return loss / num_block

In [26]:
def build_mask(seq_len):
    mask = []
    for i, s in enumerate(seq_len):
        s_mask = np.zeros((124, 1))
        s_mask[:int(s)] = np.ones((int(s), 1))
        mask += [s_mask]
    return mask

In [68]:
def train(layer_type, batch_size=64, num_epoch=100):
    
    if layer_type == 'lstm':
        embedding_layer = EmbeddingLayer(200, 240, 'lstm')
    elif layer_type == 'cnn':
        pass
        
    optimizer = torch.optim.Adam(embedding_layer.parameters(), lr=0.005)
    
    qids = list(train_idx_set.keys())
    num_batch = len(qids) // batch_size
    
    for epoch in range(1, num_epoch + 1):
        
        for batch_idx in range(1, num_batch + 1):
            
            batch_x_qids = qids[batch_idx * batch_size: (batch_idx + 1) * batch_size]
            start = time.time()
            print ('processing batch {}'.format(batch_idx))
            padded_batch_x, seq_len = process_batch(batch_x_qids, train_idx_set)
            print ('processing batch costs:', time.time() - start)
            embedding_layer.hidden = embedding_layer.init_hidden(padded_batch_x.shape[0])
            optimizer.zero_grad()
            
            start = time.time()
            qs = Variable(torch.FloatTensor(padded_batch_x))
            embeddings = embedding_layer(qs, seq_len)
            print ('embedding costs:', time.time() - start)
            start = time.time()
            
            print ('accumulating loss costs:', time.time() - start)
            loss = criterion(embeddings)
            print ('-------------------------------')
            print ('epoch:{}/{}, batch:{}/{}, loss:{}'.format(epoch, num_epoch, batch_idx, num_batch, loss.data[0]))
            print ('-------------------------------')
            start = time.time()
            loss.backward()
            print ('backprop costs:', time.time() - start)
            optimizer.step()

In [69]:
train('lstm', num_epoch=1)

processing batch 1
processing batch costs: 0.830629825592041
embedding costs: 14.15979790687561
accumulating loss costs: 0.0
-------------------------------
epoch:1/1, batch:1/198, loss:0.999708354473114
-------------------------------
backprop costs: 21.249325275421143
processing batch 2
processing batch costs: 1.7010469436645508
embedding costs: 14.345101833343506
accumulating loss costs: 9.5367431640625e-07
-------------------------------
epoch:1/1, batch:2/198, loss:0.9993116855621338
-------------------------------
backprop costs: 17.478061199188232
processing batch 3
processing batch costs: 1.0079529285430908
embedding costs: 9.926378965377808
accumulating loss costs: 9.5367431640625e-07
-------------------------------
epoch:1/1, batch:3/198, loss:0.9964604377746582
-------------------------------
backprop costs: 14.163267135620117
processing batch 4
processing batch costs: 1.2058498859405518
embedding costs: 20.66390609741211
accumulating loss costs: 9.5367431640625e-07
--------

backprop costs: 23.66404891014099
processing batch 30
processing batch costs: 1.5806190967559814
embedding costs: 12.456974983215332
accumulating loss costs: 0.0
-------------------------------
epoch:1/1, batch:30/198, loss:0.8005266785621643
-------------------------------
backprop costs: 16.482194185256958
processing batch 31
processing batch costs: 0.9081857204437256
embedding costs: 14.906774997711182
accumulating loss costs: 3.0994415283203125e-06
-------------------------------
epoch:1/1, batch:31/198, loss:0.7624027729034424
-------------------------------
backprop costs: 16.6115620136261
processing batch 32
processing batch costs: 1.6157338619232178
embedding costs: 14.416923999786377
accumulating loss costs: 1.9073486328125e-06
-------------------------------
epoch:1/1, batch:32/198, loss:0.7573146224021912
-------------------------------
backprop costs: 12.165377140045166
processing batch 33
processing batch costs: 0.9758009910583496
embedding costs: 9.29592490196228
accumula

KeyboardInterrupt: 